In [2]:
""" Default stuff for Dn quiver. """

import sympy as sp
from queue import Queue
from collections import defaultdict


class Quiver:
    def __init__(self, Q_0: list[str], Q_1: list[tuple], 
                 tau: list[tuple]=None) -> None:
        self.vertices = Q_0
        self.vertices.sort()
        self.arrows = defaultdict(list)
        for a, b in Q_1:
            self.arrows[a].append(b)
        for a in self.arrows:
            self.arrows[a].sort()
        self.translations = tau

    def __repr__(self) -> str:
        repr = "-" * 20 + "Q_0" + "-" * 20 + "\n"
        repr += ', '.join(self.vertices) + "\n\n"
        repr += "-" * 20 + "Q_1" + "-" * 20 + "\n"
        for a in self.arrows:
            repr += f"{a} -> " + ', '.join(self.arrows[a]) + "\n"

        return repr
    
    def __getitem__(self, key: str) -> list[str]:
        return self.arrows.get(key, [])
    
    def __iter__(self):
        for v in self.vertices:
            yield v


def ar_translate(M: sp.Matrix) -> sp.Matrix:
    """ Auslander-Reiten translate of a module. """
    return A * M

def l_simplex(l: int, n: int) -> set:
    """ l-simplex of dimension n for an integer l. """
    D = set()
    q = Queue()
    q.put([None] + [0] * n)
    while not q.empty():
        L = q.get()
        if sum(L[1:]) <= l:
            D.add(tuple(L[1:]))
            for i in range(1, len(L)):
                L[i] += 1
                q.put(L.copy())
                L[i] -= 1
    return D

# D_n quiver
n = 4
Q_0 = [f"{x}" for x in range(1, n + 1)]
Q_1 = [
    (f"{i}",f"{i - 1}") for i in range(2, n)
] + [(f"{n}",f"{n - 2}")]
Q = [Quiver(Q_0, Q_1)]

# Indecomposable projectives
P = [None] * (n + 1)
for i in range(1, n):
    P[i] = sp.Matrix([1] * i + [0] * (n - i))
P[n] = sp.Matrix([1] * (n - 2) + [0, 1])

# Cartan matrix
C = sp.Matrix(
    [P[i].T for i in range(1, len(P))]
).T
# Indecomposable injectives
I = [None] + [
    C.row(i-1).T for i in range(1, len(P))
]
# Coxeter matrix
A = -C.T * C**-1

# Maximum number of translations of injectives
l = [None] + [0] * (len(I) - 1)
for i in range(1, len(I)):
    M = I[i]
    while M not in P:
        M = ar_translate(M)
        l[i] += 1

In [19]:
""" Calculating D_n^1 quiver. """

# l-simplices for indecomposable injectives
D = [None] + [
    l_simplex(l[i], 1) for i in range(1, len(I))
]
# canonical basis
e = [None] + [
    sp.Matrix([0] * (i - 1) + [1] + [0] * (n - i)) for i in range(1, n + 1)
]
# some other basis
v = [None] * (n + 1)
for i in range(1, n + 1):
    if i == 1:
        v[i] = -e[i]
    else:
        v[i] = e[i - 1] - e[i]
# vertices
Q_0 = [
    f"{x}{''.join(f'{y}' for y in l)}" for x in Q[0] for l in D[int(x)]
]
# arrows


Matrix([
[ 1],
[-1],
[ 0],
[ 0]])